In [2]:
import requests
import json
import os



In [3]:
advertisers = [
"614186325698777"
]

In [4]:

def get_next_page(response):
    paging = response.get("paging", {})
    next_url = paging.get("next", None)
    return next_url


In [5]:

def main(api_url):
    all_data = []

    while api_url:
        response = requests.get(api_url)
        if response.status_code == 200:
            json_data = response.json()
            data = json_data.get("data", [])
            all_data.extend(data)
            api_url = get_next_page(json_data)
        else:
            print("Error:", response.status_code)
            break

    return all_data

In [14]:
output_dir = "custom_ads"  # Directory to save the JSON files


for id in advertisers:
    fields = "id,ad_snapshot_url,ad_creation_time,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,ad_delivery_start_time,ad_delivery_stop_time,bylines,currency,delivery_by_region,demographic_distribution,estimated_audience_size,impressions,languages,page_id,page_name,publisher_platforms,spend,target_locations,target_gender,target_ages,eu_total_reach,beneficiary_payers,age_country_gender_reach_breakdown"
    # supported_countries = ['CZ']
    limit = '100'
    start_date = '2024-01-01'
    ad_type = 'ALL'
    access_token = "EAALnc8im5MUBO0fdvFfWGLBzrHRXmnlg4GJbSZAxiHmDHd5cZAupnfxcOCZC2Jmm1HYWTjd8Rk1kQuOoZC86uur7n8zhrtIfntx0xjVOZACfXArwnu700VOZBOFze4Xsuw5ui6LqbSXPTyNs9UPZB0nbfcu1PgJjb147aMMTUjv5jZB0O3JMJ56AZCG3bD8nnwqjiiEQOsVeT"
    api_url = f"https://graph.facebook.com/v17.0/ads_archive?search_terms=&ad_type={ad_type}&ad_delivery_date_min={start_date}&ad_reached_countries=['CZ']&access_token={access_token}&unmask_removed_content=true&search_page_ids={id}&fields={fields}&limit={limit}]"
    extracted_data = main(api_url)

    # Write the extracted data to a JSON file
    filename = os.path.join(output_dir, f"data_{id}.json")
    with open(filename, "w", encoding='utf-8') as json_file:
                json.dump(extracted_data, json_file, indent=4, ensure_ascii=False)

    print(f"All Extracted Data written to {filename}")


All Extracted Data written to custom_ads\data_614186325698777.json


In [15]:
import os
import json

def replace_id_with_underscore_id(json_data):
    if isinstance(json_data, dict):
        for key, value in list(json_data.items()):
            if key == "id":
                json_data["_id"] = json_data.pop("id")
            replace_id_with_underscore_id(value)
    elif isinstance(json_data, list):
        for item in json_data:
            replace_id_with_underscore_id(item)

def process_json_file(file_path):
    with open(file_path, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)

    replace_id_with_underscore_id(data)

    with open(file_path, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

def replace_id_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            process_json_file(file_path)

if __name__ == "__main__":
    replace_id_in_folder(output_dir)


### push data to mongo

update if exists

insert if not


In [16]:
from pymongo import MongoClient

# Connection to MongoDB
client = MongoClient('mongodb://localhost:27017')

# Select your database
db = client['meta_ads_db']

# Select your collection
collection = db['meta_ads_collection']

# Loop through each file in the folder
for filename in os.listdir(output_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(output_dir, filename)

        # Load data from JSON file with 'utf-8' encoding
        with open(file_path, encoding='utf-8') as file:
            data = json.load(file)

        # Upsert each document from the JSON file
        for document in data:
            result = collection.update_one(
                {"_id": document["_id"]},
                {"$set": document},
                upsert=True
            )
        # Print the total number of documents added or updated for the current file
        print(f"File: {filename}, Total documents processed: {len(data)}")


# Close the connection
client.close()

File: data_614186325698777.json, Total documents processed: 61


## drop entire db

In [ ]:
from pymongo import MongoClient

# Connection to MongoDB
client = MongoClient('mongodb://localhost:27017')

# Select your database
db = client['meta_ads_db']

# Select your collection
collection = db['meta_ads_collection']

# Truncate (remove all documents from) the collection
result = collection.delete_many({})

# Print the number of deleted documents
print(f"Number of documents deleted: {result.deleted_count}")

# Close the connection
client.close()
